In [6]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Bidirectional, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Precision, Recall
from sklearn.metrics import classification_report
import numpy as np
import pickle

# Load the data
file_paths = [
    'Labeled data/bag_toss_labeled_1.csv',
    'Labeled data/sitting_up_down_labeled_2.csv',
    'Labeled data/stairs_labeled_4.csv',
    'Labeled data/walking_labeled_3.csv',
    'Labeled data/standing_labeled_5.csv'
]
data_frames = [pd.read_csv(file_path) for file_path in file_paths]
combined_data = pd.concat(data_frames)

# Preprocess the data
data_for_model = combined_data.drop(columns=['timestamp'])  # Drop timestamp
X = data_for_model.drop(columns=['label'])
y = data_for_model['label'].values

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

encoder = LabelEncoder()
y_encoded = encoder.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_encoded, test_size=0.2, random_state=42)

X_train_lstm = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test_lstm = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))


# Build the LSTM model
model = Sequential([
    Bidirectional(LSTM(64, return_sequences=True), input_shape=(X_train_lstm.shape[1], X_train_lstm.shape[2])),
    Dropout(0.5),
    LSTM(32),
    Dropout(0.5),
    Dense(32, activation='relu'),
    Dense(len(np.unique(y_encoded)), activation='softmax')
])

optimizer = Adam(learning_rate=0.001)
batch_size = 64

model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
#model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy', Precision(), Recall()])
#model.compile(optimizer=optimizer, 
#              loss='sparse_categorical_crossentropy', 
#              metrics=['accuracy', Precision(average='micro'), Recall(average='micro')])
#model.compile(optimizer=optimizer, 
#              loss='sparse_categorical_crossentropy', 
#              metrics=['accuracy', Precision(), Recall()])




early_stopping = EarlyStopping(monitor='val_loss', patience=5)

# Train the model
# model.fit(X_train_lstm, y_train, epochs=50, validation_split=0.2, callbacks=[early_stopping], batch_size=64)

history = model.fit(X_train_lstm, y_train, epochs=50, validation_split=0.2, callbacks=[early_stopping], batch_size=batch_size)

# Evaluate the model using the test set and print precision, recall, and F1-score
y_pred = model.predict(X_test_lstm)
y_pred_classes = np.argmax(y_pred, axis=1)

# Ensure target_names are in string format
target_names = [str(class_name) for class_name in encoder.classes_]

print(classification_report(y_test, y_pred_classes, target_names=target_names))

#print(classification_report(y_test, y_pred_classes, target_names=encoder.classes_))

# Save the model and preprocessors

model.save('LSTM_model_V2.keras')
with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)
with open('encoder.pkl', 'wb') as f:
    pickle.dump(encoder, f)

print("Model and preprocessors saved successfully.")


Epoch 1/50
293/293 [==============================] - 7s 8ms/step - loss: 1.3528 - accuracy: 0.4797 - val_loss: 1.0596 - val_accuracy: 0.5698
Epoch 2/50
293/293 [==============================] - 1s 5ms/step - loss: 0.9754 - accuracy: 0.6285 - val_loss: 0.8779 - val_accuracy: 0.6480
Epoch 3/50
293/293 [==============================] - 1s 5ms/step - loss: 0.8788 - accuracy: 0.6609 - val_loss: 0.8205 - val_accuracy: 0.6527
Epoch 4/50
293/293 [==============================] - 1s 5ms/step - loss: 0.8289 - accuracy: 0.6769 - val_loss: 0.7767 - val_accuracy: 0.6894
Epoch 5/50
293/293 [==============================] - 1s 5ms/step - loss: 0.7892 - accuracy: 0.6912 - val_loss: 0.7521 - val_accuracy: 0.6905
Epoch 6/50
293/293 [==============================] - 1s 5ms/step - loss: 0.7706 - accuracy: 0.6981 - val_loss: 0.7427 - val_accuracy: 0.6941
Epoch 7/50
293/293 [==============================] - 2s 5ms/step - loss: 0.7578 - accuracy: 0.7002 - val_loss: 0.7323 - val_accuracy: 0.6944
Epoch 

c:\Users\jakok\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\jakok\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\jakok\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo